In [53]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
from importlib import reload
import sys
import os
import time
from IPython.display import clear_output, display
from tqdm import tqdm

In [16]:
import functions; functions = reload(functions)
from functions import *

In [3]:
apikey = '3CenRhJBzNqh2_C_5S38pOyt3ozLvQDm'

In [4]:
# todo
# 1. test function for daily aggregates - none/errors/edge cases/before available
# 2. design pipeline for many jsons to csv
# 3. .db maybe?
# 4. if not, make csv
# 5. update/remake .py files to download and merge missing

In [5]:
date = '2020-01-01'
url = f'https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/{date}?apiKey={apikey}'

In [6]:
response = requests.get(url)

In [7]:
response.json()

{'queryCount': 0,
 'resultsCount': 0,
 'adjusted': True,
 'status': 'OK',
 'request_id': 'a426fcea4aa5004a97f7d0e49adcb888'}

In [50]:
unauth = daily_agg('2018-01-02')
holiday = daily_agg('2020-01-01')
good = daily_agg('2020-06-01')

Attempted to request data past historical entitlements. Please upgrade your plan at https://polygon.io/pricing
No data available for 2020-01-01


In [17]:
mon = daily_agg('2024-06-03')
tue = daily_agg('2024-06-04')
wed = daily_agg('2024-06-05')

In [18]:
alljson = []
for json in [mon, tue, wed]:
    alljson.extend(json)

In [135]:
def loadtest(filename, onecol=False):
    before = datetime.now()
    
    if onecol:
        df = pd.read_csv(filename, usecols=[0])
    else:
        df = pd.read_csv(filename)

    after = datetime.now()
    print((after - before).total_seconds())

def datelist_to_df(datelist):
    before = datetime.now()
    
    alljson = []
    for i, date in enumerate(tqdm(datelist)):
        data = daily_agg(date)
        if data is not None:
            alljson.extend(data)
    
    after = datetime.now()
    totalseconds = (after - before).total_seconds()
    minutes = totalseconds // 60
    seconds = totalseconds % 60
    print(f'{len(datelist)} file requests done in {minutes} minutes and {round(seconds, 2)} seconds')
    
    return pd.DataFrame(alljson)

def datestring_from_timestamp(timestamp):
    return datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d')

In [85]:
alldates = get_trading_days()
all = datelist_to_df(alldates)